# UW Config Tool

The `uw config` tool has four modes:
- `realize` -- creates output that is as fully rendered as possible
- `compose` -- builds up a final config by repeatedly updating a base config with the contents of other configs of the same format
- `compare` -- compares two config files
- `validate` -- ensures that a given config file is structured properly

We'll focus on `realize` and `compare` CLI in the Intro, and will continue to see examples of the other tools throughout the tutorial. API examples of these tools are available from the `uwtools` __[Config notebook](https://github.com/ufs-community/uwtools/blob/main/notebooks/config.ipynb)__.

In [1]:
import os
os.chdir("../configs")

# Running Realize

* Default format is YAML when format isn't provided and cannot be deduced from the filename extension.
* Supported formats also include INI, Fortran Namelist, Bash.
* Can transform between compatible formats. All formats can be transformed to YAML.
  


<div class="alert alert-block alert-info">
<b>Tip:</b> Jupyter uses "bash magic" to run shell commands. One-liners use <b>!</b> while multi-line bash cells may use <b>%%bash</b> or <b>%%script</b> headers.
</div>


### The `--help`, `-h` flag

In [2]:
!uw config realize -h  

usage: uw config realize [-h] [--version] [--input-file PATH]
                         [--input-format {ini,nml,sh,yaml}]
                         [--update-file PATH]
                         [--update-format {ini,nml,sh,yaml}]
                         [--output-file PATH]
                         [--output-format {ini,nml,sh,yaml}]
                         [--key-path KEY[.KEY...]] [--values-needed] [--total]
                         [--dry-run] [--quiet] [--verbose]

Realize config

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --input-file PATH, -i PATH
      Path to input file (default: read from stdin)
  --input-format {ini,nml,sh,yaml}
      Input format (default: yaml)
  --update-file PATH, -u PATH
      Path to update file (default: read from stdin)
  --update-format {ini,nml,sh,yaml}
      Update format
  --output-file PATH, -o PATH
      Path to output file (default: write to stdout)
  --output-format {ini,nml,sh,yam

### Realize an input file with `--input-file`, `-i`

The most basic option. Render all expressions possible. Any values unavailable will be left as their original Jinja2 expressions.

In [3]:
!uw config realize -i ci01.yaml

a:
  pi: 3.14
b:
  foo:
    pi: 3.14
  bar:
    one: 1
    two: 2
date: '{{ cycle }}'


### Require all expressions to be rendered with `--total` flag

In [4]:
!uw config realize -i ci01.yaml --total

[2025-09-22T21:08:53]    ERROR Config could not be realized. Try with --values-needed for details.


### Check which values remain with `--values-needed` flag

In [5]:
!uw config realize -i ci01.yaml --total --values-needed

[2025-09-22T21:08:53]     INFO Keys that are complete:
[2025-09-22T21:08:53]     INFO   a
[2025-09-22T21:08:53]     INFO   a.pi
[2025-09-22T21:08:53]     INFO   b
[2025-09-22T21:08:53]     INFO   b.foo
[2025-09-22T21:08:53]     INFO   b.foo.pi
[2025-09-22T21:08:53]     INFO   b.bar
[2025-09-22T21:08:53]     INFO   b.bar.one
[2025-09-22T21:08:53]     INFO   b.bar.two
[2025-09-22T21:08:53]     INFO 
[2025-09-22T21:08:53]     INFO Keys with unrendered Jinja2 variables/expressions:
[2025-09-22T21:08:53]     INFO   date: {{ cycle }}


### Provide input or update-values via pipes

Pipe input from bash to provide those missing values. Use the `--update-format` flag to let the tool know to use the additional information. It could instead be provided by file, in which case, the `-u` flag may be used to point to that file. 

In [6]:
%%bash
args=(
    -i ci01.yaml
    --update-format yaml
)    
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

a:
  pi: 3.14
b:
  foo:
    pi: 3.14
  bar:
    one: 1
    two: 2
date: '2025-09-24 12:00:00'
cycle: 2025-09-24T12:00:00


### Output only a subsection with the `--key-path` flag

In [7]:
%%bash
args=(
    -i ci01.yaml
    --update-format yaml
    --key-path b
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

foo:
  pi: 3.14
bar:
  one: 1
  two: 2


### Output the result to a file with `--output-file`, `-o` flag

* Default is `stdout`.
* Format is assumed from the file name provided.
* Format can be explicitly stated with the `--output-format` flag


In [8]:
%%bash
args=(
    -i ci01.yaml
    --update-format yaml
    --key-path b
    -o b.yaml
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [9]:
!cat b.yaml

foo:
  pi: 3.14
bar:
  one: 1
  two: 2


### Translate formats by providing different input and output formats

* Format translations must be compatible.
* INI and Fortran Namelist both require sections with keys/value pairs (depth=2)
* Bash may only have key/value pairs (depth=1)

#### YAML -> Namelist

In [10]:
%%bash
args=(
    -i ci01.yaml 
    --update-format yaml
    --key-path b
    -o b.nml
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [11]:
!cat b.nml

&foo
    pi = 3.14
/

&bar
    one = 1
    two = 2
/


#### YAML -> INI

In [12]:
%%bash
args=(
    -i ci01.yaml 
    --update-format yaml
    --key-path b
    -o b.ini
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [13]:
!cat b.ini

[foo]
pi = 3.14

[bar]
one = 1
two = 2


#### YAML section -> bash

In [14]:
%%bash
args=(
    -i ci01.yaml 
    --update-format yaml
    --key-path b.bar
    -o b.sh
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [15]:
!cat b.sh

one=1
two=2


# Running Compare

* Given two paths, compare entries and report on diffs
* Both files must be the same format.

In [16]:
!uw config compare -h

usage: uw config compare --path1 PATH --path2 PATH [-h] [--version]
                         [--format1 {ini,nml,sh,yaml}]
                         [--format2 {ini,nml,sh,yaml}] [--quiet] [--verbose]

Compare configs

Required arguments:
  --path1 PATH
      Path to file 1
  --path2 PATH
      Path to file 2

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --format1 {ini,nml,sh,yaml}
      Format of file 1
  --format2 {ini,nml,sh,yaml}
      Format of file 2
  --quiet, -q
      Print no logging messages
  --verbose, -v
      Print all logging messages


## Compare two namelists

In [17]:
!uw config compare --path1 ci02.nml --path2 b.nml

[2025-09-22T21:08:56]     INFO - ci02.nml
[2025-09-22T21:08:56]     INFO + b.nml
[2025-09-22T21:08:56]     INFO ---------------------------------------------------------------------
[2025-09-22T21:08:56]     INFO ↓ ? = info | -/+ = line unique to - or + file | blank = matching line
[2025-09-22T21:08:56]     INFO ---------------------------------------------------------------------
[2025-09-22T21:08:56]     INFO   bar:
[2025-09-22T21:08:56]     INFO     one: 1
[2025-09-22T21:08:56]     INFO -   three: 3
[2025-09-22T21:08:56]     INFO     two: 2
[2025-09-22T21:08:56]     INFO   foo:
[2025-09-22T21:08:56]     INFO -   pi: 3.1415
[2025-09-22T21:08:56]     INFO ?           --
[2025-09-22T21:08:56]     INFO +   pi: 3.14


### No diffs

In [18]:
!uw config compare --path1 b.ini --path2 b.ini

[2025-09-22T21:08:56]     INFO - b.ini
[2025-09-22T21:08:56]     INFO + b.ini


In [19]:
!uw config compare --path1 b.ini --path2 b.nml

[2025-09-22T21:08:56]    ERROR Formats do not match: ini vs nml
